In [1]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data", one_hot=True)

/home/foriequal0/.local/share/virtualenvs/tf-tutorial-pgL6-UTG/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
learning_rate = 0.01
num_steps = 1000
batch_size = 128
display_step = 100

n_input = 28*28
n_classes = 10
dropout = 0.75

sess = tf.Session()

dataset = tf.data.Dataset.from_tensor_slices((mnist.train.images, mnist.train.labels))
dataset = dataset.batch(batch_size)
iterator = dataset.make_initializable_iterator()

sess.run(iterator.initializer)

X, Y = iterator.get_next()

In [3]:
def conv_net(x, n_classes, dropout, reuse, is_training):
    with tf.variable_scope('ConvNet', reuse=reuse):
        x = tf.reshape(x, shape=[-1, 28, 28, 1])

        conv1 = tf.layers.conv2d(x, 32, 5, activation=tf.nn.relu)
        conv1 = tf.layers.max_pooling2d(conv1, 2, 2)

        conv2 = tf.layers.conv2d(conv1, 64, 3, activation=tf.nn.relu)
        conv2 = tf.layers.max_pooling2d(conv2, 2, 2)

        fc1 = tf.contrib.layers.flatten(conv2)

        fc1 = tf.layers.dense(fc1, 1024)
        fc1 = tf.layers.dropout(fc1, rate=dropout, training=is_training)

        out = tf.layers.dense(fc1, n_classes)
        out = tf.nn.softmax(out) if not is_training else out

    return out

logits_train = conv_net(X, n_classes, dropout, reuse=False, is_training=True)
logits_test = conv_net(X, n_classes, dropout, reuse=True, is_training=False)

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits_train, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

correct_pred = tf.equal(tf.argmax(logits_test, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [4]:
init = tf.global_variables_initializer()
sess.run(init)

for step in range(1, num_steps + 1):
    try:
        sess.run(train_op)
    except tf.errors.OutOfRangeError:
        sess.run(iterator.initializer)
        sess.run(train_op)
        
    if step % display_step == 0 or step == 1:
        # Calculate batch loss and accuracy
        # (note that this consume a new batch of data)
        loss, acc = sess.run([loss_op, accuracy])
        print("Step {}, loss={:4f}, acc={:.3f}".format(step, loss, acc))

print("Optimization Finished!")
sess.close()

Step 1, loss=5.869912, acc=0.078
Step 100, loss=0.335803, acc=0.938
Step 200, loss=0.249090, acc=0.945
Step 300, loss=0.243536, acc=0.945
Step 400, loss=0.157495, acc=0.977
Step 500, loss=0.183098, acc=0.969
Step 600, loss=0.220469, acc=0.953
Step 700, loss=0.223666, acc=0.953
Step 800, loss=0.152564, acc=0.945
Step 900, loss=0.100603, acc=0.984
Step 1000, loss=0.275790, acc=0.969
Optimization Finished!
